In [2]:
from typing import List, Tuple, Dict

import numpy as np

from sympy import symbols, Matrix, Symbol
from sympy.matrices import zeros
from sympy.core import relational

from decimal import Decimal

In [10]:
def eqns_to_matrix(
    eqns: List[relational.Relational], dependents: List[Symbol]
) -> Tuple[Matrix, Matrix, Matrix]:
    """Converts list of relations to slack variable matrix and vector formalism.
    
    The result relates to the original eqn such that `m.deps + v + s= 0`
    
    For example [a1 * x <= b1, a2 * x >= b2] becomes
    
    m = | a1 |  and v = | b1 | s = | - s1 |
        | a2 |          | b2 |     | + s2 |
    
    Arguments:
        eqns: List of relational equations. LHS must be linear in dependents, RHS constant.
        dependents: List of dependent variables


    Returns:
        m and v, here m is the Matrix and v the vector containing the slack variable
    
    """
    dependents = set(dependents)

    mat_coeffs = set()
    vec_coeffs = set()
    for eqn in eqns:
        mat_coeffs = mat_coeffs.union(eqn.lhs.free_symbols.difference(set(dependents)))
        vec_coeffs = vec_coeffs.union(eqn.rhs.free_symbols.difference(set(dependents)))

    n_deps = len(dependents)
    n_eqns = len(eqns)

    s_vars = symbols(f"s(1:{n_eqns+1})")

    v = Matrix([[-eqn.rhs] for eqn in eqns])
    s = Matrix([[s if isinstance(eqn, relational.LessThan) else -s] for s, eqn in zip(s_vars, eqns)])
    m = zeros(rows=n_eqns, cols=n_deps)
    for ne, eqn in enumerate(eqns):
        for nd, dep in enumerate(dependents):
            m[ne, nd] = eqn.lhs.coeff(dep)

    return m, v, s

In [11]:
def rescale_expressions(expr: Symbol, subs: Dict[str, str]) -> Symbol:
    """Rescales and substitutes all values. 
    
    The values are multiplied by 10**power such that all values are integers.
    
    Arguments:
        expr: The expression to substitute
        subs: The symbol to value map. Must be strings.
        
    Returns:
        The rescaled and substituded expression
    """
    max_neg_power = 0
    for par, val in subs.items():
        exponent = Decimal(val).as_tuple().exponent
        max_neg_power = exponent if exponent < max_neg_power else max_neg_power

    fact = 10 ** (-max_neg_power)

    print(f"Multipying by {fact}")

    rescaled_subs = {par: int(Decimal(val) * fact) for par, val in subs.items()}

    return expr.subs(rescaled_subs)

In [12]:
def int_to_bitarray(i: int, bits=8) -> np.ndarray:
    """Converts an integer to an array where each value corresponds to a bit
    
    Arguments:
        i: The integer
        bits: The available bits for the substitutin.
        
    Returns:
        An array of ones and zeros. First element is smallest number
    """
    bit_string = (f"{{0:0{bits}b}}").format(i)
    if len(bit_string) > bits:
        raise ValueError(f"{i} is too large to be represented by {bits} bits")
    return np.array([int(ii) for ii in bit_string[::-1]])

In [6]:
a1, a2, b1, b2, x = symbols("a1, a2, b1, b2, x")
subs = {"a1": "0.456", "a2": "0.1", "b1": "10", "b2": "-2.1"}

eqns = [
    a1 * x <= b1,
    a2 * x >= b2,
]

In [7]:
m, v, s = eqns_to_matrix(eqns, [x])
deps = Matrix([x])

mm = m @ deps + v + s
mm

Matrix([
[a1*x - b1 - s1],
[a2*x - b2 - s2]])

In [8]:
print(subs)

rescale_expressions(mm, subs)

{'a1': '0.456', 'a2': '0.1', 'b1': '10', 'b2': '-2.1'}
Multipying by 1000


Matrix([
[-s1 + 456*x - 10000],
[ -s2 + 100*x + 2100]])

In [9]:
bits = 8
max_bit = 2 ** bits - 1
print(f"using {bits} bits ranging from 0 ... {max_bit}")


int_to_bitarray(1, 8)

using 8 bits ranging from 0 ... 255


array([1, 0, 0, 0, 0, 0, 0, 0])

The idea is as follows: we have a list of equations which is of the form
\begin{equation}
    \vec f( \vec x, \vec s ) = A \cdot \vec x + \vec b + C \vec s = \vec 0 \, ,
\end{equation}
Where $C$ is diagonal and entries are $c_{ii} \in \{-1, 0, 1 \}$ (corresponding to the constraint type). 

This can be rewritten as
\begin{equation}
    \vec f( \vec \xi ) = \alpha \cdot \xi + \vec b = \vec 0 \, ,
\end{equation}
where $\vec \xi = (\vec x, \vec s)$ and thus
\begin{equation}
    \alpha = \begin{pmatrix} A & 0 \\ 0 & C \end{pmatrix} \, , \qquad
\end{equation}

However, our variables $\vec x$ and $\vec s$ are in a bit representation
\begin{equation}
   q = \sum_{i=0}^{N_b-1} \psi_i^{(q)} 2^i \, , \qquad \psi_i^{(q)} \in \{0, 1 \} \, .
\end{equation}

Thus our vector $ \vec \xi $ corresponds to a vector of size $N_b \times (N_x + N_s)$ in the qbit basis
\begin{equation}
    \vec \psi_\xi
    =
    \left(
        \psi_0^{(x_1)}, \cdots, \psi_{N-1}^{(x_1)},
        \cdots,
        \psi_0^{(x_{N_x})}, \cdots, \psi_{N-1}^{(x_{N_x})},
        \psi_{0}^{(s_1)}, \cdots, \psi_{N_s-1}^{(s_1)},
        \psi_{0}^{(s_{N_s})}, \cdots, \psi_{N_s-1}^{(s_{N_S})},
    \right) \, .
\end{equation}

We are interested in a map $Q$ such that $\vec \xi = Q \vec \psi_\xi$.
Thus $Q$ is a diagonal and it's entries are blockwise

\begin{equation}
    Q
    =
    (\tilde Q^{(x_1)}, \cdots, \tilde Q^{(x_{N_x})}, \tilde Q^{(s_1)}, \cdots \tilde Q^{(s_{N_s})}) \, ,
    \qquad
    \tilde{Q}^{(i)}_n
        = \begin{cases}
        (1, 2, 2^2, \cdots, 2^{N-1}) \cdot \vec e_n & n \in \text{range of (i)} \\
        0 & \text{otherwise}
      \end{cases}
    \, .
\end{equation}

As an example, let's pick $N_x =1$, $N_b=2$ and $N_s =3$, thus we have

\begin{equation}
    Q
    =
    \begin{pmatrix}
        1 & 2 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 2 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 1 & 2 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 2
    \end{pmatrix}
    \, , \qquad
    \begin{pmatrix}
        x_1 \\ s_1 \\ s_2 \\ s_3
    \end{pmatrix}
    =
    Q \cdot
    \begin{pmatrix}
        \psi_0^{(x_1)} \\ \psi_1^{(x_1)} \\
        \psi_0^{(s_1)} \\ \psi_1^{(s_1)} \\
        \psi_0^{(s_2)} \\ \psi_1^{(s_2)} \\
        \psi_0^{(s_3)} \\ \psi_1^{(s_3)} 
    \end{pmatrix}
    =
    \begin{pmatrix}
        \psi_0^{(x_1)} + 2 \psi_1^{(x_1)} \\
        \psi_0^{(s_1)} + 2 \psi_1^{(s_1)} \\
        \psi_0^{(s_2)} + 2 \psi_1^{(s_2)} \\
        \psi_0^{(s_3)} + 2 \psi_1^{(s_3)} 
    \end{pmatrix}
\end{equation}

In [44]:
def get_bit_map(nvars: int, nb: int) -> np.ndarray:
    """
    """
    bitmap = 2 ** np.arange(nb)
    q = np.zeros([nvars, nb*nvars], dtype=int)
    for n in range(nvars):
        q[n, n * nb : ((n + 1) * nb)] = bitmap

    return q

In [46]:
get_bit_map(3, 4)

array([[1, 2, 4, 8, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 2, 4, 8, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 4, 8]])

Finally, our matrix $M$ just depends on $x$, while $C$ just depends on $s$. So the EQN becomes

\begin{equation}
    \vec f( \vec x, \vec s )
    = 
    \alpha Q \vec \psi_\xi + \vec b     
\end{equation}

Finally we want to solve

\begin{equation}
    \vec x_0
    =
    \text{argmax}_{\vec x}\left(\max_{\vec s}\left( 
        \vec d \cdot \vec x + p \vec f( \vec x, \vec s ) \cdot \vec f ( \vec x, \vec s ) 
    \right)\right)
\end{equation}
note that the solutions are most likely degenerate since we might find vectors for same $x$ but different $s$ which fulfill the above equation.

in the new basis this becomes

\begin{equation}
    \psi_{\xi_0}
    =
    \text{argmax}_{\vec \psi_{\xi}}\left( 
        \vec \delta \cdot \vec \psi_{\xi} 
        +  p (\alpha Q \vec \psi_\xi  + \vec b)\cdot(\alpha Q \vec \psi_\xi  + \vec b)  
    \right)
    =
    \text{argmax}_{\vec \psi_{\xi}}\left( 
        \vec \psi_{\xi} \cdot M_Q \cdot \vec \psi_{\xi}
    \right)
\end{equation}

\begin{equation}
    M_Q 
    = 
    \begin{pmatrix}
        \text{diag}(\vec d) & 0 \\ 
        0 & 0
    \end{pmatrix}Q 
    + p Q^T \alpha^T \alpha Q
    + {\color{red}\vec b \cdot \vec b}
\end{equation}
